In [1]:
import csv
import os
import pickle
import re
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pywt
import seaborn as sns
from keras.utils import to_categorical
from scipy.integrate import simps
from scipy.signal import butter, filtfilt, iirnotch, welch
from tqdm import tqdm

from Brain_to_Image import batch_csv as batch
from Brain_to_Image import helper_functions as hf
from Brain_to_Image.dataset_formats import (MDB2022_MNIST_EP_params,
                                            MDB2022_MNIST_IN_params,
                                            MDB2022_MNIST_MU_params,
                                            keys_MNIST_EP, keys_MNIST_IN,
                                            keys_MNIST_MU)

#%matplotlib widget
sns.set(font_scale=1.2)
print(os.getcwd())

dataset = "MNIST_EP"
root_dir = f"Datasets/MindBigData MNIST of Brain Digits/{dataset}"
if True:
    # ## TRAIN
    input_file = f"train_MindBigData2022_{dataset}.csv"
    output_file = f"train_MindBigData2022_{dataset}.pkl"
else:
    ## TEST
    input_file = f"test_MindBigData2022_{dataset}.csv"
    output_file = f"test_MindBigData2022_{dataset}.pkl"

label = 'digit_label'
## MNIST_MU sf = 220, 440 samples , MNIST_EP sf = 128, 256 samples , MNIST_IN sf = 128, 256 samples
if "_EP" in dataset or "_IN" in dataset:
    sample_rate = 128  #Hz
else:
    sample_rate = 220  #Hz
# Define notch frequencies and widths
notch_freqs = [50] #, 60]  # Line noise frequencies (50 Hz and harmonics)
notch_widths = [1] #, 2]  # Notch widths (in Hz)
# Define filter parameters
lowcut = 0.4 # 0.4  # Low-cutoff frequency (Hz)
highcut = 60 # 110  # High-cutoff frequency (Hz)
class_labels = [0,1,2,3,4,5,6,7,8,9]
keys_ = ['EEGdata_T7','EEGdata_P7','EEGdata_T8','EEGdata_P8']

c:\Users\timta\Documents\Msc Notes\CMP9140-2324 Research Project


The MBD2022 data set raw csv files are loaded and processed into pandas dataframes and saved as pickle format.
- one time process to make data more accessible for use in other processing

In [ ]:
## Process the raw data file from the hugging face download site.
# df = batch.batch_process_csv_pandas(f"{root_dir}/{input_file}",f"{root_dir}/{output_file}",MBD=MDB2022_MNIST_EP_params)
# df.info()
# df['digit_label'].value_counts()

Use this block to read the raw data from pandas pickle file or insert filtered_ prefix to read filtered data

In [ ]:
## Raw
if True:
    print(f"** reading file {root_dir}/{output_file}")
    df = pd.read_pickle(f"{root_dir}/{output_file}")
    df = df[df[label]!=-1]
    df.info()
    print(df[label].value_counts())
## Filtered
if False:
    print(f"** reading file {root_dir}/filtered_bp_corr_{output_file}") #filtered_{output_file}")
    df = pd.read_pickle(f"{root_dir}/filtered_bp_corr_{output_file}") #filtered_{output_file}")
    df = df[df[label]!=-1]
    df.info()
    print(df[label].value_counts())
    print(df.columns)

if False:
    features = np.load(f"{root_dir}/training_{dataset}_corr_93_signals.npy")
    labels = np.load(f"{root_dir}/training_{dataset}_corr_93_signals_labels.npy")

In [ ]:
print(f"** reading file {root_dir}/mne_epoch_rejection_idx.pkl")
epoched_indexs = pd.read_pickle(f"{root_dir}/mne_epoch_rejection_idx.pkl")
df = df.loc[epoched_indexs['passed']]
print(df[label].value_counts())

This section is for sampling from the total data:
- remove the -1 labelled data which is the blank image
- groupby the lable
- sample using the fraction.

In [ ]:
fraction = 1
sampled_indexes = df.groupby(label).apply(lambda x: x.sample(frac=fraction)).index.get_level_values(1).tolist()
sampled_df = df.loc[sampled_indexes]
#sampled_df.info()
df = None
print(sampled_df[label].value_counts())
print(sampled_df[label].value_counts().sum())

In [5]:
# function to Create windowed data based on window 32, overlap 4 = step size 28
def sliding_window_eeg(signal, window_size=32, overlap=4):
    """
    Apply a sliding window with overlap to a 2-second EEG signal.

    Parameters:
    signal (numpy.ndarray): 1D array of EEG signal data (256 samples)
    window_size (int): Size of each window (default: 32)
    overlap (int): Number of overlapping samples between windows (default: 4)

    Returns:
    numpy.ndarray: 2D array of windowed data
    """
    if len(signal) != 256:
        raise ValueError("Signal length must be 256 samples (2 seconds at 128Hz)")

    # Calculate the step size
    step = window_size - overlap

    # Calculate the number of windows
    num_windows = (len(signal) - window_size) // step + 1

    # Create an empty array to store the windowed data
    windowed_data = np.zeros((num_windows, window_size, 1))

    # Apply the sliding window
    for i in range(num_windows):
        start = i * step
        end = start + window_size
        windowed_data[i] = signal[start:end].reshape(window_size,1)
    return windowed_data

# w_data = sliding_window_eeg(df[df[label]==3].iloc[4]['EEGdata_AF3'],16,2)
# w_data.shape

In [ ]:
record = 15608
signal = sampled_df.iloc[record][['digit_label']+keys_MNIST_EP[8:12]][keys_MNIST_EP[8]]
w_data = sliding_window_eeg(signal)
w_data.shape

fig, axs = plt.subplots(2, 1, figsize=(16, 15))
fig.suptitle(f"Record {record}: class label {sampled_df.iloc[record][['digit_label']]}")
w_data = w_data.reshape(9,32)
axs[0].plot(signal)
axs[0].set_title('Original Data')
axs[0].set_xlabel('Time')
axs[0].set_ylabel('Amplitude')
for i in range(w_data.shape[0]):
    axs[1].plot(w_data[i],linewidth=0.5)
axs[1].plot(np.mean(w_data, axis=0),linewidth=2.5)
axs[1].set_title('Windowed Data')

The block is used to test downsampling with a convolution.

In [ ]:
record = 1319
series = sampled_df.iloc[record][['digit_label']+keys_MNIST_EP[8:12]]
# Define a kernel (filter) of size 8
kernel = np.ones(8) / 8  # Simple averaging filter
# Function to apply convolution to a single series (pandas Series)
def apply_convolution(series, kernel, stride=8):
    # Perform convolution
    convolved = np.convolve(series, kernel, mode='valid')
    # Downsample to simulate stride
    return pd.Series(convolved[::stride])

series = series[keys_MNIST_EP[6:10]].apply(apply_convolution, kernel=kernel, stride=8)
series.head()

In [ ]:
record = 5082

# inspect signal data from raw
# psd_keys = [f"{key}_PSD" for key in keys_]
# fig = hf.plot_4_signals(sampled_df.iloc[record][['digit_label']+psd_keys],
#                     f"RAW EEG series signal for class {sampled_df.iloc[record]['digit_label']}"
#                     ,y_labels=psd_keys,
#                     digit_label=label,
#                     norm=False,
#                     fs=sample_rate,
#                     x_units="sec",
#                     y_units="muV")
# inspect signal data from sampled or downsampled raw data

fig = hf.plot_4_signals(sampled_df.iloc[record][['digit_label']+keys_MNIST_EP[8:12]],
                    f"RAW EEG series signal for class {sampled_df.iloc[record]['digit_label']}"
                    ,y_labels=keys_MNIST_EP[8:12],
                    digit_label='digit_label',
                    norm=True,
                    fs=sample_rate,
                    x_units="sec",
                    y_units="muV")

#fig = hf.plot_time_sequence(df.iloc[record]['EEGdata_T7_PSD'],title=f"Class {df.iloc[record][label]}")
plt.show()

Apply notchfilter and bandpass filters to raw/sampled raw data. 
* No need to run this block if loaded the filtered data.

In [ ]:
df_copy = sampled_df.copy() # sampled_df.copy()
apply_filter = True
for key in keys_MNIST_EP:
    df_copy[f"{key}_Freq"] = pd.NA
    df_copy[f"{key}_PSD"] = pd.NA

for idx, row in tqdm(sampled_df.iterrows()):
    if apply_filter:

        filtered = hf.apply_notch_filter(row[keys_MNIST_EP],sample_rate,notch_freqs=notch_freqs,notch_widths=notch_widths)
        filtered = hf.apply_butterworth_filter(filtered,sample_rate,lowcut,highcut,order=5)

    else:
        #filtered = row[keys_MNIST_IN]
        filtered = hf.apply_notch_filter(row[keys_MNIST_EP],sample_rate,notch_freqs=notch_freqs,notch_widths=notch_widths)
    window_length = 1 * sample_rate    # 2 seconds
    for key in keys_MNIST_EP:
        freq, PSD = welch(filtered[key], fs=sample_rate, nperseg=window_length)
        filtered[f"{key}_Freq"] = freq
        filtered[f"{key}_PSD"] = PSD
    filtered["digit_label"] = row['digit_label']
    df_copy.loc[idx] = filtered

df_copy.info()

calculate the correlation coefficents

In [ ]:
def corr(arr, corr_arr):
    return np.corrcoef(arr, corr_arr)

def subtract_series(arr, subtrahend):
    return arr - subtrahend

from scipy.stats import pearsonr
example_data = sampled_df.iloc[record][keys_MNIST_EP]
#print(type(example_data))
#example_data['label'] = 0
arr = np.stack(example_data.values)
car = np.mean(arr, axis=0) 
baseline = np.mean(car[:16])
car_corrected = car - baseline

## subtract CAR from each channel
example_car = example_data.apply(lambda x: subtract_series(x, car_corrected)) 

correlations_1 = [pearsonr(car_corrected, example_data[i])[0] for i in keys_MNIST_EP[8:12]]

correlations_3 = [pearsonr(car_corrected, example_car[i])[0] for i in keys_MNIST_EP[8:12]]

correlations_2 = [np.corrcoef(example_data[i], car_corrected)[0][1] for i in keys_MNIST_EP[8:12]]

fig, axs = plt.subplots(5, 1, figsize=(16, 15))

# Plot original data
axs[0].plot(np.array([np.linspace(0,256,256) for _ in range(4)]),np.stack(example_data[keys_MNIST_EP[8:12]]))
axs[0].set_title('Original Data')
axs[0].set_xlabel('Time')
axs[0].set_ylabel('Amplitude')

# Plot CAR-referenced data
axs[1].plot(np.array([np.linspace(0,256,256) for _ in range(4)]),np.stack(example_car[keys_MNIST_EP[8:12]].values))
axs[1].set_title('CAR-Referenced Data')
axs[1].set_xlabel('Time')
axs[1].set_ylabel('Amplitude')

# Plot correlation coefficients
axs[2].bar(range(len(keys_MNIST_EP[8:12])), correlations_1)
axs[2].set_title('Correlation Coefficients with CAR')
axs[2].set_xlabel('Channel')
axs[2].set_ylabel('Correlation Coefficient')
axs[2].set_xticks(range(len(keys_MNIST_EP[8:12])))
axs[2].set_xticklabels([f'Ch{i}' for i in keys_MNIST_EP[8:12]])

# # Plot correlation coefficients
axs[3].bar(range(len(keys_MNIST_EP[8:12])), correlations_3)
axs[3].set_title('Correlation Coefficients with CAR')
axs[3].set_xlabel('Channel')
axs[3].set_ylabel('Correlation Coefficient')
axs[3].set_xticks(range(len(keys_MNIST_EP[8:12])))
axs[3].set_xticklabels([f'Ch{i}' for i in keys_MNIST_EP[8:12]])

# # Plot correlation coefficients
axs[4].bar(range(len(keys_MNIST_EP[8:12])), correlations_2)
axs[4].set_title('Correlation Coefficients with CAR')
axs[4].set_xlabel('Channel')
axs[4].set_ylabel('Correlation Coefficient')
axs[4].set_xticks(range(len(keys_MNIST_EP[8:12])))
axs[4].set_xticklabels([f'Ch{i}' for i in keys_MNIST_EP[8:12]])

plt.tight_layout()
plt.show()



In [ ]:
example_data['label'] = 0
example_car['label'] = 0
fig = hf.plot_4_signals(example_data[['label']+keys_MNIST_EP[8:12]],f"RAW EEG series signal for class {sampled_df.iloc[record]['digit_label']}"
                    ,y_labels=keys_MNIST_EP[8:12],
                    digit_label='digit_label',
                    norm=True,
                    fs=sample_rate,
                    x_units="sec",
                    y_units="muV")

fig = hf.plot_4_signals(example_car[['label']+keys_MNIST_EP[8:12]],f"RAW EEG series signal for class {sampled_df.iloc[record]['digit_label']}"
                    ,y_labels=keys_MNIST_EP[8:12],
                    digit_label='digit_label',
                    norm=True,
                    fs=sample_rate,
                    x_units="sec",
                    y_units="muV")

In [6]:
def calculate_snr(signal_reduced, signal_sample):
    """
    Calculate the Signal-to-Noise Ratio (SNR) given a noise-reduced signal and a sample signal.

    Parameters:
    - signal_reduced: ndarray of the noise-reduced signal.
    - signal_sample: ndarray of the sample signal.

    Returns:
    - snr: The Signal-to-Noise Ratio in decibels (dB).
    """
    # Calculate the noise as the difference between the sample and noise-reduced signal
    noise = signal_sample - signal_reduced

    # Calculate the power of the signal (mean squared value)
    signal_power = np.mean(signal_reduced ** 2)

    # Calculate the power of the noise (mean squared value)
    noise_power = np.mean(noise ** 2)

    # Calculate the SNR in decibels (dB)
    snr = 10 * np.log10(signal_power / noise_power)

    return snr

In [ ]:
snr = []
for i in range(len(keys_MNIST_EP[8:12])):
    x = example_car[keys_MNIST_EP[8:12]][i]
    s = example_data[keys_MNIST_EP[8:12]][i]
    snr.append(calculate_snr(x,s))
    
print(np.mean(snr))

In [7]:
for class_label in class_labels:
    class_df = sampled_df[sampled_df[label]==class_label]
    snr = []
    for idx, row in tqdm(class_df.iterrows()):
        for key in keys_MNIST_EP[8:12]:
            x = row[key] - (row[key] -row['erp'])
            s = row[key]
            snr.append(calculate_snr(x,s))
    print(f"Class {class_label}: mean SNR {np.mean(snr):.3f}")

1177it [00:00, 5929.96it/s]


Class 0: mean SNR 0.378


1086it [00:00, 5905.16it/s]


Class 1: mean SNR 0.521


1132it [00:00, 5049.00it/s]


...

Class 7: mean SNR 0.299


1113it [00:00, 6173.64it/s]


Class 8: mean SNR 0.382


1148it [00:00, 5838.24it/s]

Class 9: mean SNR 0.280


In [9]:
#sampled_df = None
#sampled_df = df_copy.copy()
df_copy = None

In [ ]:
df_copy = sampled_df.copy()

for key in keys_MNIST_EP:
    df_copy[f"{key}_corr"] = pd.NA
df_copy[f"erp"] = pd.NA

for idx, row in tqdm(sampled_df.iterrows()):
    corr_data = row[keys_MNIST_EP]
    arr = np.stack(corr_data.values)
    # Calculate ERP
    car = np.mean(arr, axis=0)
    # Baseline correction (using first 125 ms as baseline)
    baseline = np.mean(car[:16])
    car_corrected = car - baseline

    for key in keys_MNIST_EP:
        #car_subtracted = corr_data[key] - car_corrected
        #correlation = np.corrcoef(car_subtracted, car_corrected)[0, 1]
        correlation = np.corrcoef(corr_data[key], car_corrected)[0, 1]
        row[f"{key}_corr"] = correlation
    #corr_data[label] = row[label]
    row['erp'] = car_corrected

    df_copy.loc[idx] = row

df_copy.info()


In [ ]:

sampled_df = None
df_copy['digit_label'].value_counts()

In [ ]:
corr_keys_ = ['EEGdata_T7_corr','EEGdata_P7_corr','EEGdata_T8_corr','EEGdata_P8_corr']
df_copy['corr_mean_core'] = df_copy[corr_keys_].mean(axis=1)
corr_keys_ = [f"{key}_corr" for key in keys_MNIST_EP]
df_copy['corr_mean_all'] = df_copy[corr_keys_].mean(axis=1)
#df_copy.dropna(axis=1, inplace=True)
df_copy.info()
print(df_copy.columns)

In [ ]:
df_copy['corr_mean_core'].mean()

In [12]:
fraction = 1
sampled_indexes = df_copy[df_copy['corr_mean_core'] > 0.2].groupby(label).apply(lambda x: x.sample(frac=fraction)).index.get_level_values(1).tolist()
sampled_df = df_copy.loc[sampled_indexes]
#sampled_df.info()
print(sampled_df[label].value_counts())
print(sampled_df[label].value_counts().sum())

7    1195
5    1178
0    1177
6    1169
9    1148
2    1132
8    1113
3    1102
4    1096
1    1086
Name: digit_label, dtype: int64
11396


In [ ]:

from sklearn.model_selection import train_test_split


feature_data = []
label_data = []
for class_label in class_labels:
    class_df = sampled_df[sampled_df[label]==class_label]
    for idx, row in tqdm(class_df.iterrows()):
        for key in keys_:
            w_data = sliding_window_eeg(row[key])
            feature_data.append(np.array(w_data))
            label_data.append(to_categorical(int(class_label),num_classes=len(class_labels)))

train_data = np.array(feature_data)
labels = np.array(label_data).astype(np.uint8)

print(train_data.shape)
print(labels.shape)

x_train, x_test, y_train, y_test = train_test_split(train_data, labels, test_size=0.1, random_state=42)




In [ ]:
#data_out = {'x_train':x_train,'x_test':x_test,'y_train':y_train,'y_test':y_test}
prefix = ["data_4ch_90_32_4_","fil_corr_","data_4ch_epoch_filtered_324_0-2","data_4ch_epoch_filtered_324_0-85"]
print(f"writing {root_dir}/{prefix[3]}{output_file}")
data_out = {'x_train':x_train,'x_test':x_test,'y_train':y_train,'y_test':y_test} #{'x_test':train_data,'y_test':labels}
with open(f"{root_dir}/{prefix[3]}{output_file}", 'wb') as f:
    pickle.dump(data_out, f)

Save/load the filtered/sampled/downsampled data to file, change the prefix as required.

In [ ]:
## CHANGE FILE NAME FOR TEST / TRAIN
prefix = ["filtered_bp_corr_","fil_corr_","epoch_filtered_","epoch_filtered_-car_","epoch_filtered_rcar_"]
print(f"writing {root_dir}/{prefix[4]}{output_file}")  # filtered_{output_file}
with open(f"{root_dir}/{prefix[4]}{output_file}", 'wb') as f:
    pickle.dump(df_copy, f)

In [11]:
prefix = ["filtered_bp_","fil_corr_","epoch_filtered_","epoch_filtered_-car_","epoch_filtered_rcar_"]
print(f"** reading file {root_dir}/{prefix[3]}{output_file}")
df_copy = pd.read_pickle(f"{root_dir}/{prefix[3]}{output_file}")
df_copy.info()
print(df_copy[label].value_counts())
print(df_copy.columns)

** reading file Datasets/MindBigData MNIST of Brain Digits/MNIST_EP/epoch_filtered_-car_train_MindBigData2022_MNIST_EP.pkl
<class 'pandas.core.frame.DataFrame'>
Int64Index: 38509 entries, 16632 to 28018
Data columns (total 60 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   digit_label       38509 non-null  int64  
 1   EEGdata_AF3       38509 non-null  object 
 2   EEGdata_AF4       38509 non-null  object 
 3   EEGdata_F7        38509 non-null  object 
 4   EEGdata_F8        38509 non-null  object 
 5   EEGdata_F3        38509 non-null  object 
 6   EEGdata_F4        38509 non-null  object 
 7   EEGdata_FC5       38509 non-null  object 
 8   EEGdata_FC6       38509 non-null  object 
 9   EEGdata_T7        38509 non-null  object 
 10  EEGdata_T8        38509 non-null  object 
 11  EEGdata_P7        38509 non-null  object 
 12  EEGdata_P8        38509 non-null  object 
 13  EEGdata_O1        38509 non-null  object 
 14  EEGdata

USe to display/inspect signal data after filtering.

In [ ]:
df_copy = sampled_df.copy()


In [ ]:
record = 1010


fig = hf.plot_4_signals(df_copy.iloc[record][['digit_label']+keys_],
                    f"RAW EEG series signal for class {df_copy.iloc[record]['digit_label']}"
                    ,y_labels=keys_,
                    digit_label='digit_label',
                    norm=False,
                    fs=sample_rate,
                    x_units="sec",
                    y_units="muV")
plt.show()

#fig = hf.plot_time_sequences(df_copy.iloc[record][keys_+['erp']])

In [ ]:
freq,psd = welch(df_copy.iloc[record]['EEGdata_T8'], fs=sample_rate, nperseg=32, noverlap=4)
fig, axs = plt.subplots(1, 1, figsize=(16, 4), sharex=True)
#axs.plot(freq,psd)
axs.semilogy(freq,psd)
#axs.psd(df_copy.iloc[record]['EEGdata_T8'], NFFT=64, Fs=sample_rate, scale_by_freq=True)
axs.grid(True)
fig.tight_layout()
plt.show()

In [ ]:
# Create the figure and subplots
r, c = 2, 2
fig, axs = plt.subplots(r, c, figsize=(16, 8), sharex=True)
fig.suptitle(f"Power signal for class {df_copy.iloc[record][label]}", fontsize=16)
cnt = 0

for i in range(r):
    for j in range(c):

        #freq,psd = welch(filtered[cnt], fs=sample_rate, nperseg=window_length) #64 #, noverlap=32)
        #freq = df_copy.iloc[record][f"{keys_[cnt]}_Freq"]
        #psd = df_copy.iloc[record][f"{keys_[cnt]}_PSD"]
        axs[i,j].plot(freq,psd)
        axs[i,j].grid(True)
        axs[i,j].set_title(f"{keys_[cnt]}", fontsize=12)
        cnt += 1
fig.tight_layout()
plt.show()

In [ ]:

sample_size = 200
random_state = 23
train_data = []
label_data = []


In [ ]:
# Define a kernel (filter) of size 8
kernel = np.ones(8) / 8  # Simple averaging filter
use_downsample = False
slice = 0 #(64,192)
## raw signals
# sub_df = df[df['digit_label']!= -1]
## filtered signals
#sub_df = df_copy[df_copy['digit_label']!= -1]

for idx, record in tqdm(df_copy.iterrows()):
    label_data.append(to_categorical(record[label],num_classes=len(class_labels)))
    selected_features = []
    #key_features = []
    for key in keys_:
        if use_downsample:
            downsample = np.convolve(record[f"{key}"], kernel, mode="valid")
            downsample = downsample[::8]
            #selected_features.append(record[f"{key}"])    ## each sample is ~8ms at 128 Hz @2 seconds = 256 samples, drop the first ~250msec = 30 samples
            selected_features.append(downsample)
        else:
            if slice == 0:
                selected_features.append(record[f"{key}"])    ## each sample is ~8ms at 128 Hz @2 seconds = 256 samples, drop the first ~250msec = 30 samples
            else:
                selected_features.append(record[f"{key}"][slice[0]:slice[1]])

    #selected_features = StandardScaler().fit_transform(selected_features)
    #train_data.append(np.array(selected_features).reshape(len(selected_features),len(record[f"{key}"]),1))
    train_data.append(np.array(selected_features).T)

train_data = np.array(train_data)
label_data = np.array(label_data).astype(np.uint8)


In [ ]:
print(train_data.shape)
print(label_data.shape)

np.save(f"{root_dir}/training_{dataset}_corr_93T_signals",train_data)
np.save(f"{root_dir}/training_{dataset}_corr_93T_signals_labels",label_data)

# np.save(f"{root_dir}/testing_MNIST_EP_signals",train_data)
# np.save(f"{root_dir}/testing_MNIST_EP_signals_labels",label_data)